In [1]:
# 심층 신경망
# 인공 신경망에서 층을 더 많이 넣은게 심층 신경망이다.

# 기본적인 베이스
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.linear_model import SGDClassifier
from tensorflow import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()
train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)

4431872/4422102 [==============================] - 0s 0us/step


In [2]:
# 이제 여기서 "은닉층 100개의 뉴런", 그리고 거기에 활성화 함수를 넣어보자.
# 입력층과 출력층 사이에 있는 모든 층을 은닉층이라고 부른다.

# 출력층에서 사용하는 활성화 함수는 이진인 경우, 다중인 경우에 쓰이는게 딱 나눠져 있어서
# 제한이 되어있는데, 은닉층에서는 이게 비교적 자유롭다. 예를 들어 볼 렐루 함수라는걸 사용한다.

# 참고로 회귀로 신경망 쓸거면 활성화 함수를 사용하지 않아도 된다.
# 분류는, 확률을 출력하기 위해서 사용하는 것 이므로.

# 그러면 왜 은닉층에도 활성화 함수가 필요할까?
# 선형 방정식 두개를 생각해보자. 단순 선형 계산만으로 이루어져 있다면,
# 변수 하나를 소거할 수 있게 된다. 즉, 의미가 없는 층이 된다는 것이다.
# 그래서 첫 층에서 출력된 값을 적당히 비틀어 줄 필요가 있다.
 
# 은닉층이 몇 개의 뉴런을 가져야 할지 판단하는건 경험이 필요하다. (적어도 출력층 뉴런보다는 많아야 한다.)
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))

# 얘는 출력층이다.
dense2 = keras.layers.Dense(10, activation='softmax')

In [4]:
# 다음으로 심층 신경망을 연결해서 다 써서 만들어보자.
model = keras.Sequential([dense1, dense2])
model.summary()

# 그렇다면 두번째 층의 모델 파라미터의 총 개수는? -> 1,010개이다. 100(은닉층) * 10(출력층) + 10(출력층마다 있는 절편)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [7]:
# keras.Sequential에 층을 추가하는 다른 방법.
model = keras.Sequential([
                          keras.layers.Dense(100, activation='sigmoid', input_shape=(784,),
                                             name='hidden'),
                          keras.layers.Dense(10,activation='softmax',name='output')
],name='패션 MNIST 모델')

# 이렇게 하면 층들을 한 눈에 보기도 쉽고 이름까지 지정할 수 있다. 층 이름은 반드시 영문!!
model.summary()



Model: "패션 MNIST 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [9]:
# 또 다른 방법
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10,activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 100)               78500     
                                                                 
 dense_3 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [10]:
# 이제 훈련해보자.
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.5346 - accuracy: 0.8151
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3923 - accuracy: 0.8572
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3607 - accuracy: 0.8698
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3394 - accuracy: 0.8768
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3235 - accuracy: 0.8825


# 렐루 함수

In [16]:
# 렐루 함수
# max(0,z)이다. 음수로 가는건 그냥 0으로 만들어 버린다.
# 이미지 처리에서 아주 좋은 성능을 낼 수 있다.

# 그 전에 지금까지 reshape 써서 1차원으로 펼쳤던거 말고 
# 케라스에서 제공하는 Flatten 층을 공부해보자.

model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28))) # 여기서 일자로 펴줬다!
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

model.summary()
# 그렇다고 Flatten 층을 학습 층의 개수에 추가해주진 않는다.
# 그러나 summary()에서 기존의 차원을 엿볼 수 있다.
# 전처리까지 모델에 넣어보자 하는게 케라스의 철학이다.

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()
train_scaled = train_input / 255.0
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=10)

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_5 (Flatten)         (None, 784)               0         
                                                                 
 dense_14 (Dense)            (None, 100)               78500     
                                                                 
 dense_15 (Dense)            (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5270 - accuracy: 0.8152
Epoch 2/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3894 - accuracy: 0.8597
Epoch 3/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3543 - accuracy: 0.8730
Epoch 4/

In [17]:
# 이제 최종 점수는!
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 2s 3ms/step - loss: 0.4003 - accuracy: 0.8775


[0.4002610743045807, 0.8774999976158142]

## 옵티마이저
* 지금까지는 하이퍼파라미터를 잘 다뤘는데 이제 나옴.
----
추가할 은닉층의 개수  
뉴런 개수  
활성화 함수  
층의 종류  
배치 사이즈 매개변수  
에포크 매개변수  


In [19]:
# 케라스는 기본적으로 미니배치 경사 하강법을 사용한다.
# 미니배치 개수는 기본적으로 32개다.

# 옵티마이저 - 케라스는 다양한 종류의 경사 하강법 알고리즘을 제공
sgd = keras.optimizers.SGD(learning_rate=0.1) # 이거 'sgd'로 넣는거랑 똑같다. 학습률도 조정할 수 있다.
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy')

# 기본 경사 하강법 옵티마이저는 모두 SGD 클래스에서 제공하는데, 해당 클래스의 momentum 매개변수의 기본값은 0이다.
# 이를 0보다 큰 값으로 지정하면 마치 이전의 그레이던트를 가속도처럼 사용하는 모멘텀 최적화를 사용한다.
# 보통 0.9 이상으로 지정한다.

# 아래처럼 네스테로프 모멘텀 최적화를 키면 모멘텀 최적화를 2번 반복하여 구현하는데, 대부분 이게 기본보다 좋다.
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)

In [20]:
# 모델이 최적점에 가까이 갈수록 학습률을 낮추는 것을 '적응적 학습률'이라고 한다.
# Adagrad, RMSprop이 있는데, optimizer 매개변수의 기본값이 rmsprop이다.
'''
adagrad = keras.optimizers.Adagrad()
model.compile(optimizer=adagrad, loss='sparse_categorical_crossentropy', metrics='accuracy')
'''

'''
rmsprop = keras.optimizers.RMSprop()
model.compile(optimizer=rmsprop, loss='sparse_categorical_crossentropy', metrics='accuracy')
'''

model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28))) # 여기서 일자로 펴줬다!
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.5232 - accuracy: 0.8173
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3897 - accuracy: 0.8587
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3486 - accuracy: 0.8731
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3259 - accuracy: 0.8813
Epoch 5/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3060 - accuracy: 0.8885


In [21]:
# 아담을 사용한 최종 결과
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 2ms/step - loss: 0.3588 - accuracy: 0.8705


[0.3588126599788666, 0.8705000281333923]